# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Configure Logger

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')

#### Generate a list of shipment numbers with the check digit calculation

In [ ]:
from scrabu import generate_shipment_numbers #340434188193324407|340434188193323500

#### Download the HTML content for a list of shipment numbers

In [ ]:
from scrabu import request

#### Parsing HTML and converting it into JSON

In [ ]:
from scrabu import html_to_json

#### Structure the shipment details in JSON format

In [ ]:
from scrabu import shipment_details

In [ ]:
from scrabu import save_dictionary

#### This function summarizes all previous steps for a single shipment number

In [ ]:
from scrabu import process_shipment_number

#### Multi-threading

In [ ]:
from scrabu import main

In [ ]:
if __name__ == '__main__':
    main(shipment_number=340434188193324407, size=10000, max_workers=4)